### DO NOT RUN ALL

The model creation cell takes half an hour to run.

Instead, just begin by loading the oosdf.csv and taking it from there (the model is there)

**Student Name: Adrian Lechter**

### Do you want to run with oosdf?

yes or no?


In [1]:
Answer = 'Yes'

In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [3]:
#Import Libs
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from IPython.display import display, HTML, display_html, Markdown
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import zscore

if Answer == 'Yes':
    print('Running with OOSDF')
    
else:
    print('Running without OOSDF')

Running with OOSDF


In [4]:
if Answer == 'Yes':
    
    #Import dataset
    display((Markdown('<span style="color:black;font-size:15px"> **' 
        + str('Import') + '**</span>')))
    df_import = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv")
    #Inspect dataset
    display(df_import.head())
    
    #Import oosdf_import dataset
    display((Markdown('<span style="color:black;font-size:15px"> **' 
        + str('Model from CSV Import') + '**</span>')))
    df_model = pd.read_csv('model.csv', header = 0, names = ['y', 'pred'])
    display(df_model.shape)
    display(df_model.columns)
    display(df_model.head())
    display(df_model.describe())
    display(df_model.info())
#     display((Markdown('<span style="color:black;font-size:15px"> **' 
#         + str('Final Model') + '**</span>')))
#     display(model.shape)
#     display(model.columns)    
#     display(model.head())
#     display(model.describe())
#     display(model.info())
    
else:
    #Import dataset
    df_import = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv")

    #Inspect dataset
    display(df_import.columns)
    display(df_import.head())
    
#     df.columns = ['oostarget', 'oosprediction']

<span style="color:black;font-size:15px"> **Import**</span>

,id,convention,height,max,cat2,number,usage,region,length,code,power,item,weight,country,target
0,1,CO-1A,4284.51,44907,CA-E,16669,US-7,RE-4,12471.1127,CO-B,27351.36,IT-17,13722,CO-1,44098.106769
1,2,CO-C,806.88,48831,CA-A,8652,US-20,RE-15,10035.7085,CO-E,42323.89,IT-1E,33779,CO-0,95567.294044
2,3,CO-19,2859.80,40760,CA-16,23103,US-17,RE-1D,14442.6566,CO-5,30660.91,IT-14,26633,CO-23,48583.507153
3,4,CO-2B,5823.87,33597,CA-9,17680,US-10,RE-1D,15121.4937,CO-B,59456.24,IT-8,14537,CO-11,130572.202064
4,5,CO-5,NaN,29848,CA-9,24136,US-21,RE-4,18093.9147,CO-4,46998.44,IT-5,21135,CO-1E,85768.812850


<span style="color:black;font-size:15px"> **Model from CSV Import**</span>

(10809, 2)

Index(['y', 'pred'], dtype='object')

,y,pred
0,44098.106769,44080.312
1,130572.202064,130472.440
2,67926.242813,67547.000
3,64558.309366,64520.047
4,48666.213736,48268.867


,y,pred
count,10809.000000,10809.000000
mean,89338.872653,89360.762609
std,37692.449571,37622.328655
min,-50194.241772,-32202.014000
25%,63668.960703,63614.547000
50%,89186.892747,89256.590000
75%,115138.736504,115078.445000
max,239516.024243,232187.340000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10809 entries, 0 to 10808
Data columns (total 2 columns):
y       10809 non-null float64
pred    10809 non-null float64
dtypes: float64(2)
memory usage: 169.0 KB


None

### Data pre-processing
- Normalize all numerics to z-scores and all text/categoricals to dummies. Do not normalize the target. Your target (y) is the field named target.
- If you find any missing values (NA's), replace them with the median values for that column.
- Seperate target and response variables

In [ ]:
# Encode the feature vector
ids = df_import['id']
df_import.drop('id',1,inplace=True)

#Fill missing height values with median
df_import['height'] = df_import['height'].fillna(value = df_import['height'].median())

#Fill missing max values with median
df_import['max'] = df_import['max'].fillna(value = df_import['max'].median())

#Fill missing number values with median
df_import['number'] = df_import['number'].fillna(value = df_import['number'].median())

#Fill missing length values with median
df_import['length'] = df_import['length'].fillna(value = df_import['length'].median())

#Fill missing power values with median
df_import['power'] = df_import['power'].fillna(value = df_import['power'].median())

#Fill missing weight values with median
df_import['weight'] = df_import['weight'].fillna(value = df_import['weight'].median())

# Generate dummies for convention
df_import = pd.concat([df_import,pd.get_dummies(df_import['convention'],prefix="convention")],axis=1)
df_import.drop('convention', axis=1, inplace=True)

# Standardize height
df_import['height_standardized'] = zscore(df_import['height'])
df_import.drop('height', axis=1, inplace=True)

# Standardize max
df_import['max_standardized'] = zscore(df_import['max'])
df_import.drop('max', axis=1, inplace=True)

# Generate dummies for cat2
df_import = pd.concat([df_import,pd.get_dummies(df_import['cat2'],prefix="cat2")],axis=1)
df_import.drop('cat2', axis=1, inplace=True)

# Standardize number
df_import['number_standardized'] = zscore(df_import['number'])
df_import.drop('number', axis=1, inplace=True)

# Generate dummies for usage
df_import = pd.concat([df_import,pd.get_dummies(df_import['usage'],prefix="usage")],axis=1)
df_import.drop('usage', axis=1, inplace=True)

# Generate dummies for region
df_import = pd.concat([df_import,pd.get_dummies(df_import['region'],prefix="region")],axis=1)
df_import.drop('region', axis=1, inplace=True)

# Standardize length
df_import['length_standardized'] = zscore(df_import['length'])
df_import.drop('length', axis=1, inplace=True)

# Generate dummies for code
df_import = pd.concat([df_import,pd.get_dummies(df_import['code'],prefix="code")],axis=1)
df_import.drop('code', axis=1, inplace=True)

# Standardize power
df_import['power_standardized'] = zscore(df_import['power'])
df_import.drop('power', axis=1, inplace=True)

# Generate dummies for item
df_import = pd.concat([df_import,pd.get_dummies(df_import['item'],prefix="item")],axis=1)
df_import.drop('item', axis=1, inplace=True)

# Standardize weight
df_import['weight_standardized'] = zscore(df_import['weight'])
df_import.drop('weight', axis=1, inplace=True)

# Generate dummies for country
df_import = pd.concat([df_import,pd.get_dummies(df_import['country'],prefix="country")],axis=1)
df_import.drop('country', axis=1, inplace=True)

#Inspect dataset
df_import.head()

In [ ]:
# Convert to numpy - Classification
x_columns = df.columns.drop('target')
x = df[x_columns].values
y = df['target'].values

### Model creation
- Use a 5-fold cross-validation and return out of sample predictions. 

The code snippit below takes >30min to run as a heads up

In [ ]:
kf = KFold(5, shuffle=True, random_state=42) # Use for KFold classification
    
oos_y = []
oos_pred = []

fold = 0
for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,epochs=500)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)    

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")

# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")    
    
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )

In [ ]:
#Save the oosDF to csv so I dont need to run this again
# oosDF.to_csv('oosdf.csv',index=False)

In [ ]:
# model = pd.concat( [oos_y, oos_pred],axis=1 )
# model.to_csv('model.csv',index=False)
model.head()

### Final Submission

From the [assignment instructions]('https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb'):

_"Your submission should contain the id (column name id), your prediction (column name pred"), the expected value (from the reg-33-data.csv dataset, named y, and the absolute value of the difference between the expected and predicted (column name diff*)."_

In [6]:
#Extract ids
print("ID columns")
df_ids = df_import['id'].to_frame()
display(df_ids.head())

#Extract y and oosprededict
print("Pred and y columns")
display(df_model.head())

#Create final submission by joining the df_ids and df_model dataset
print("Joining them")
df_final_submission = df_ids.join(df_model)

df_final_submission['diff'] = abs(df_final_submission['y']- df_final_submission['pred'])
display(df_final_submission.head())


#Need to move the column names


ID columns


,id
0,1
1,2
2,3
3,4
4,5


Pred and y columns


,y,pred
0,44098.106769,44080.312
1,130572.202064,130472.440
2,67926.242813,67547.000
3,64558.309366,64520.047
4,48666.213736,48268.867


Joining them


,id,y,pred,diff
0,1,44098.106769,44080.312,17.794769
1,2,130572.202064,130472.440,99.762064
2,3,67926.242813,67547.000,379.242813
3,4,64558.309366,64520.047,38.262366
4,5,48666.213736,48268.867,397.346736


In [7]:
#Submitting Assignment

# This is your student key that I emailed to you at the beginnning of the semester.
key = ""  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/assignment_yourname_class5.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class5.ipynb'  # Windows
file ='/Users/adrianlechter/assignment_alechter_class5.ipynb'  # Mac/Linux

# os.chdir(file)

oosDF = df_final_submission

# # Begin assignment
df_import = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv")

# # Continue here

submit(source_file=file,data=oosDF,key=key,no=5)

Success: Submitted Assignment #5 for adrian:
This is your first submission of this assignment.
No warnings on your data. You will probably do well, but no guarantee. :-)



### Checking Submission

In [8]:

import pandas as pd
import base64
import os

def list_submits(key):
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
                      headers={'x-api-key': key},
                      json={})
    if r.status_code == 200:
        print("Success: \n{}".format(r.text))
    else:
        print("Failure: {}".format(r.text))

def display_submit(key,no):
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
                      headers={'x-api-key': key},
                      json={'assignment':no})
    if r.status_code == 200:
        print("Success: \n{}".format(r.text))
    else:
        print("Failure: {}".format(r.text))

In [10]:
# Show a listing of all submitted assignments.
key = ""
display(list_submits(key))

# Show one assignment, by number.
display(display_submit(key,5))

Success: 
Assignment #5: Submitted 1 times, last on: 2020-01-10T21:08:52.507Z



None

Success: 
Assignment #5: Submitted 1 times, last on: 2020-01-10T21:08:52.507Z
*** Check ***
No warnings on your data. You will probably do well, but no guarantee. :-)



None